In [10]:
# Cell 1 - Import necessary libraries
import requests
import pandas as pd

# Base URL for PokeAPI
BASE_URL = "https://pokeapi.co/api/v2/"

def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None


In [11]:
charmander = requests.get(BASE_URL + 'pokemon/charmander')

In [2]:
# Function to fetch berry-specific data
def get_berry_data():
    url = BASE_URL + "berry/"
    berry_data = []
    
    while url:
        data = fetch_data(url)
        if data:
            for berry in data['results']:
                berry_info = fetch_data(berry['url'])
                if berry_info:
                    # Append berry-specific information to berry_data
                    berry_entry = {
                        'id': berry_info['id'],
                        'name': berry_info['name'],
                        'growth_time': berry_info['growth_time'],
                        'max_harvest': berry_info['max_harvest'],
                        'natural_gift_power': berry_info['natural_gift_power'],
                        'size': berry_info['size'],
                        'smoothness': berry_info['smoothness'],
                        'soil_dryness': berry_info['soil_dryness'],
                        'firmness': berry_info['firmness']['name'],
                        'item': berry_info['item']['name'],
                        'natural_gift_type': berry_info['natural_gift_type']['name']
                        
                    }
                    berry_data.append(berry_entry)
            # Update the URL for the next page of results
            url = data['next']
        else:
            break
    
    # Create a DataFrame for berries
    berry_df = pd.DataFrame(berry_data).set_index('id')
    return berry_df

# Example usage
berry_df = get_berry_data()
berry_df


,name,growth_time,max_harvest,natural_gift_power,size,smoothness,soil_dryness,firmness,item,natural_gift_type
id,,,,,,,,,,
1,cheri,3,5,60,20,25,15,soft,cheri-berry,fire
2,chesto,3,5,60,80,25,15,super-hard,chesto-berry,water
3,pecha,3,5,60,40,25,15,very-soft,pecha-berry,electric
4,rawst,3,5,60,32,25,15,hard,rawst-berry,grass
5,aspear,3,5,60,50,25,15,super-hard,aspear-berry,ice
...,...,...,...,...,...,...,...,...,...,...
60,enigma,24,5,80,155,60,7,hard,enigma-berry,bug
61,micle,24,5,80,41,60,7,soft,micle-berry,rock
62,custap,24,5,80,267,60,7,super-hard,custap-berry,ghost


In [3]:
# Function to fetch flavor-specific data
def get_flavor_data():
    url = BASE_URL + "berry/"
    flavor_data = []
    
    while url:
        data = fetch_data(url)
        if data:
            for berry in data['results']:
                berry_info = fetch_data(berry['url'])
                if berry_info:
                    # Append flavor information to flavor_data
                    for flavor in berry_info['flavors']:
                        flavor_entry = {
                            'berry_id': berry_info['id'],  # Link to the berry
                            'flavor_name': flavor['flavor']['name'],
                            'flavor_potency': flavor['potency']
                        }
                        flavor_data.append(flavor_entry)
            # Update the URL for the next page of results
            url = data['next']
        else:
            break
    
    # Create a DataFrame for flavors
    flavor_df = pd.DataFrame(flavor_data)
    return flavor_df

# Example usage
flavor_df = get_flavor_data()
flavor_df


,berry_id,flavor_name,flavor_potency
0,1,spicy,10
1,1,dry,0
2,1,sweet,0
3,1,bitter,0
4,1,sour,0
...,...,...,...
315,64,spicy,10
316,64,dry,0
317,64,sweet,0
318,64,bitter,0


In [4]:
# Merge the berries and flavors dataframes on berry_id (id in berry_df, berry_id in flavor_df)
merged_df = pd.merge(berry_df, flavor_df, left_index=True, right_on='berry_id')

# For each berry, find the row with the highest potency
max_potency_df = merged_df.loc[merged_df.groupby('berry_id')['flavor_potency'].idxmax()]

# Now, we have a DataFrame with each berry and its corresponding highest potency flavor
# We can assign the flavor with the highest potency back to berry_df
berry_df['highest_potency_flavor'] = max_potency_df.set_index('berry_id')['flavor_name']
berry_df['highest_potency_value'] = max_potency_df.set_index('berry_id')['flavor_potency']

# Display the updated berry_df with the highest potency flavor and potency value
berry_df

,name,growth_time,max_harvest,natural_gift_power,size,smoothness,soil_dryness,firmness,item,natural_gift_type,highest_potency_flavor,highest_potency_value
id,,,,,,,,,,,,
1,cheri,3,5,60,20,25,15,soft,cheri-berry,fire,spicy,10
2,chesto,3,5,60,80,25,15,super-hard,chesto-berry,water,dry,10
3,pecha,3,5,60,40,25,15,very-soft,pecha-berry,electric,sweet,10
4,rawst,3,5,60,32,25,15,hard,rawst-berry,grass,bitter,10
5,aspear,3,5,60,50,25,15,super-hard,aspear-berry,ice,sour,10
...,...,...,...,...,...,...,...,...,...,...,...,...
60,enigma,24,5,80,155,60,7,hard,enigma-berry,bug,spicy,40
61,micle,24,5,80,41,60,7,soft,micle-berry,rock,dry,40
62,custap,24,5,80,267,60,7,super-hard,custap-berry,ghost,sweet,40


In [5]:
# Function to fetch berry and flavor-specific data
def get_berry_and_flavor_data():
    url = BASE_URL + "berry/"
    berry_flavor_data = []
    
    while url:
        data = fetch_data(url)
        if data:
            for berry in data['results']:
                berry_info = fetch_data(berry['url'])
                if berry_info:
                    # Extract berry-specific information
                    berry_entry = {
                        'id': berry_info['id'],
                        'name': berry_info['name'],
                        'growth_time': berry_info['growth_time'],
                        'max_harvest': berry_info['max_harvest'],
                        'natural_gift_power': berry_info['natural_gift_power'],
                        'size': berry_info['size'],
                        'smoothness': berry_info['smoothness'],
                        'soil_dryness': berry_info['soil_dryness'],
                        'firmness': berry_info['firmness']['name'],
                        'item': berry_info['item']['name'],
                        'natural_gift_type': berry_info['natural_gift_type']['name']
                    }
                    
                    # Initialize a dictionary to store flavor details
                    flavor_dict = {}
                    
                    # Process flavors for each berry
                    for flavor in berry_info.get('flavors', []):
                        flavor_key = f"flavor_{flavor['flavor']['name']}"  # Create a dynamic column name
                        flavor_dict[flavor_key] = flavor['potency']  # Store potency value

                    # Combine berry entry with flavor data
                    combined_entry = {**berry_entry, **flavor_dict}
                    berry_flavor_data.append(combined_entry)
            # Update the URL for the next page of results
            url = data['next']
        else:
            break
    
    # Create a DataFrame for berries and their flavors
    berry_flavor_df = pd.DataFrame(berry_flavor_data).set_index('id')
    return berry_flavor_df

# Example usage
berry_flavor_df = get_berry_and_flavor_data()
berry_flavor_df


,name,growth_time,max_harvest,natural_gift_power,size,smoothness,soil_dryness,firmness,item,natural_gift_type,flavor_spicy,flavor_dry,flavor_sweet,flavor_bitter,flavor_sour
id,,,,,,,,,,,,,,,
1,cheri,3,5,60,20,25,15,soft,cheri-berry,fire,10,0,0,0,0
2,chesto,3,5,60,80,25,15,super-hard,chesto-berry,water,0,10,0,0,0
3,pecha,3,5,60,40,25,15,very-soft,pecha-berry,electric,0,0,10,0,0
4,rawst,3,5,60,32,25,15,hard,rawst-berry,grass,0,0,0,10,0
5,aspear,3,5,60,50,25,15,super-hard,aspear-berry,ice,0,0,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,enigma,24,5,80,155,60,7,hard,enigma-berry,bug,40,10,0,0,0
61,micle,24,5,80,41,60,7,soft,micle-berry,rock,0,40,10,0,0
62,custap,24,5,80,267,60,7,super-hard,custap-berry,ghost,0,0,40,10,0
